In [1]:
import pandas as pd

In [2]:
commits = pd.read_pickle('../../data/01_data_from_db/commits.pkl')
commits.head()

,commit_id,committer_id,project_id,created_at
0,1,17,10,2012-04-08 02:29:18
1,2,18,10,2011-12-29 21:12:27
2,3,18,10,2011-12-29 21:11:10
3,4,17,10,2011-12-29 20:54:15
4,5,18,10,2011-12-29 20:29:49


In [3]:
commits['year'] = commits.created_at.dt.year
commits['month'] = commits.created_at.dt.month
commits.head()

,commit_id,committer_id,project_id,created_at,year,month
0,1,17,10,2012-04-08 02:29:18,2012,4
1,2,18,10,2011-12-29 21:12:27,2011,12
2,3,18,10,2011-12-29 21:11:10,2011,12
3,4,17,10,2011-12-29 20:54:15,2011,12
4,5,18,10,2011-12-29 20:29:49,2011,12


## Nowe commity w miesiącu

In [4]:
new_commits = commits.groupby(['project_id', 'year', 'month']).count().reset_index().rename(columns={'created_at': 'new_commits'}).drop(columns={'commit_id', 'committer_id'}, axis=1)
new_commits

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


Wersja alternatywna

In [5]:
commits.groupby(by= ['project_id', 'year', 'month'], as_index=False).agg({'created_at': pd.Series.nunique}).rename(columns={'created_at': 'new_commits'})

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


In [6]:
new_commits.to_pickle('../../data/02_prepared_data/new_commits.pkl')

## Liczba unikalnych commitujących w miesiącu

In [16]:
new_unique_committers = commits.groupby(by = ['project_id', 'year', 'month'], as_index=False).agg({'committer_id': pd.Series.nunique}).rename(columns={'committer_id': 'new_unique_committers'})
new_unique_committers

,project_id,year,month,new_unique_committers
0,2,2010,5,1
1,2,2010,6,1
2,2,2010,7,1
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,1
18758,90366,2013,9,1
18759,91413,2013,9,1
18760,91417,2013,9,1


In [8]:
new_unique_committers.to_pickle('../../data/02_prepared_data/new_unique_committers.pkl')

## (tmp) Dołączenie nowych commitów w miesiącu do tablicy projektów

> **TODO** TRZEBA ZASTANOWIĆ SIĘ, CO ZROBIĆ Z COMMITAMI, KTÓRE POWSTAŁY PRZED UTWORZENIEM PROJEKTÓW

In [9]:
projects = pd.read_pickle('../../data/01_data_from_db/projects.pkl')

In [10]:
with_new_commits = pd.merge(projects, new_commits, on='project_id', how='left', sort=False)
with_new_commits.shape

(118620, 7)

In [11]:
with_new_commits.isnull().sum()

project_id         0
name               0
language           0
created_at         0
year           99858
month          99858
new_commits    99858
dtype: int64

In [12]:
with_new_commits.dropna(how='any').shape

(18762, 7)

In [13]:
projects.sort_values(by=['created_at'])

,project_id,name,language,created_at
79168,79191,rails,Ruby,2008-02-07 23:35:21
5325,5326,beanstalkd,C,2008-03-31 20:20:41
104284,104307,paperclip,Ruby,2008-04-10 19:58:20
78830,78852,rails,Ruby,2008-04-11 02:19:47
79169,79192,rails,Ruby,2008-04-11 03:03:53
...,...,...,...,...
108669,108692,jekyll,Ruby,2013-10-05 17:16:57
108666,108689,three.js,JavaScript,2013-10-05 19:09:49
108673,108696,django,Python,2013-10-05 19:28:13
108674,108697,gitlabhq,Ruby,2013-10-05 20:02:27


In [14]:
commits.sort_values(by=['created_at'])

,commit_id,committer_id,project_id,created_at,year,month
36307,37486,25071,8955,2003-05-27 07:19:11,2003,5
36305,37484,25071,8955,2003-05-27 07:22:55,2003,5
36302,37481,25257,8955,2003-05-28 02:42:11,2003,5
36298,37477,25071,8955,2003-05-29 02:54:53,2003,5
36295,37474,25257,8955,2003-05-30 03:33:12,2003,5
...,...,...,...,...,...,...
251612,259016,172590,84407,2013-09-16 22:00:50,2013,9
245923,253314,24813,76945,2013-09-16 22:13:44,2013,9
249939,257340,168658,79159,2013-09-16 22:45:16,2013,9
251316,258720,33630,83893,2013-09-16 22:58:22,2013,9


In [15]:
for y in range(2003, 2015):
    print(f'{y}: {commits[commits.year == y].year.count().sum()}')

2003: 93
2004: 20
2005: 10
2006: 1385
2007: 1828
2008: 4465
2009: 11509
2010: 20846
2011: 60181
2012: 88913
2013: 65922
2014: 0
